In [1]:
import pandas as pd

In [2]:
crashes = pd.read_csv("../data/dwh/CrashesFact.csv")
drivers = pd.read_csv("../data/dwh/DriversFact.csv")
roaddim = pd.read_csv("../data/dwh/RoadDim.csv")
vehicles = pd.read_csv("../data/dwh/VehicleDim.csv")

In [9]:
len(drivers)

172105

In [4]:
fact_merged = drivers.merge(crashes, on='ReportNumber')

In [15]:
'2023325234'[:4]

'2023'

In [17]:
fact_merged['DateHourKey']

0         2024010314
1         2024010314
2         2023121612
3         2023122912
4         2023122912
             ...    
172100    2015070511
172101    2020091217
172102    2021070117
172103    2022111102
172104    2022121509
Name: DateHourKey, Length: 172105, dtype: int64

In [16]:
fact_merged[fact_merged['DateHourKey'].astype(str).str[:4] == 2023]

,ReportNumber,VehicleCrashKey,DriverAtFault,DriverInjurySeverity,DriverSubstanceAbuse,DriverDistractedBy,VehicleType,VehicleMovement,VehicleGoingDir,VehicleDamageExtent,...,RoadDivision,Latitude,Longitude,RoadKey,CrossStreetKey,NonMotoristTotal,NonMotoristInjury,NonMotoristFatal,DateHourKey,LocationAreaKey


In [5]:
len(fact_merged)

172105

In [9]:
fact_vehicles = fact_merged.merge(vehicles, on='VehicleKey')

In [10]:
len(fact_vehicles)

172105

In [15]:
all = fact_vehicles.merge(roaddim, left_on='RoadKey', right_on='RoadKey').merge(roaddim, left_on='CrossStreetKey', right_on='RoadKey')

In [16]:
all.columns

Index(['ReportNumber', 'VehicleID', 'DriverAtFault', 'InjurySeverity',
       'DriverSubstanceAbuse', 'DriverDistractedBy', 'VehicleType',
       'VehicleMovement', 'VehicleGoingDir', 'SpeedLimit', 'ParkedVehicle',
       'SubstanceAbuseContributed', 'VehicleKey', 'LocalCaseNumber',
       'AgencyName', 'ACRSReportType', 'Datetime', 'HitRun', 'LaneDirection',
       'LaneNumber', 'NumberOfLanes', 'RoadGrade', 'NonTraffic',
       'OffRoadIncident', 'AccidentAtFault', 'CollisionType',
       'SurfaceCondition', 'Light', 'TrafficControl', 'Junction',
       'IntersectionType', 'RoadAlignment', 'RoadCondition', 'RoadDivision',
       'Latitude', 'Longitude', 'RoadKey_x', 'CrossStreetKey',
       'NonMotoristTotal', 'NonMotoristInjury', 'NonMotoristFatal', 'Make',
       'Year', 'BaseModel', 'BodyClass', 'Cylinders', 'Displacement',
       'Transmission', 'Drivetrain', 'FuelType', 'CityMPG', 'HighwayMPG',
       'RoadName_x', 'RouteType_x', 'RoadName_y', 'RouteType_y', 'RoadKey_y'],
      

In [17]:
len(all)

172105